In [1]:
# from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

# llm = Ollama(model='mistral:instruct')
llm = ChatOllama(model='mistral:instruct')
# llm = ChatOllama(model="gemma:2b")
# llm = ChatOllama(model="gemma2:9b")
llm.invoke('<s>[INST] tell me about yourself? [/INST]')

AIMessage(content=" I am a large language model trained by Mistral AI. I was designed to assist with a wide range of tasks such as answering questions, helping generate text, summarizing information, and much more. I can process vast amounts of data and provide responses in multiple languages. My goal is to make it easier for people to find answers, create content, and understand complex topics. While I don't have personal experiences or emotions like humans, I am programmed to learn and adapt over time based on interactions with users like you.", response_metadata={'model': 'mistral:instruct', 'created_at': '2024-07-18T16:31:45.074578Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1988209292, 'load_duration': 8619875, 'prompt_eval_count': 15, 'prompt_eval_duration': 240477000, 'eval_count': 107, 'eval_duration': 1736582000}, id='run-dbdcbd1b-6f92-4794-b9a2-76e40bf1429e-0')

In [59]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [60]:
config={"configurable":{"session_id":"chat10"}}

## Scenario 1

In [61]:
from langchain_core.messages import HumanMessage,SystemMessage
response=with_message_history.invoke(
    # [
        # SystemMessage(content="<s>[INST] You are a helpful AI Bot. In a case if you don't know the answer then tell you don't know it[/INST]<s>"),
    #  HumanMessage(content="[INST]Hi , My name is Nabarun and I am a Senior Data Scientist[/INST]")],
    # [HumanMessage(content="Hi , My name is Nabarun and I am a Chief AI Engineer")],
    [   
        # SystemMessage(content="You are a helpful Chatbot who answers users request. But don't answer harmful content"),
        HumanMessage(content="Hi , My name is Nabarun and I am a Chief AI Engineer")
    ],
    config=config
)
response

AIMessage(content=" Hello Nabarun, it's nice to meet you! As a Chief AI Engineer, I imagine you have a lot of interesting work. How can I assist you today? Are there any specific questions or topics related to artificial intelligence that you would like to discuss?", response_metadata={'model': 'mistral:instruct', 'created_at': '2024-07-18T15:48:30.488546Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1172446500, 'load_duration': 13004375, 'prompt_eval_count': 22, 'prompt_eval_duration': 243184000, 'eval_count': 57, 'eval_duration': 914085000}, id='run-9674e91c-9a4c-446e-8702-8c3f23d6497e-0')

In [62]:
response=with_message_history.invoke(
    # [
        # SystemMessage(content="<s>[INST] You are a helpful AI Bot. In a case if you don't know the answer then tell you don't know it[/INST]<s>"),
        # HumanMessage(content="[INST]what is my name[/INST]")],
    # [HumanMessage(content="Hi , what is my name")],
    [   
        # SystemMessage(content="You are a helpful Chatbot who answers users request. But don't answer harmful content"),
        HumanMessage(content="Hi , what is my name")
    ],
    config=config
)
response

AIMessage(content=" Your name is Nabarun, as mentioned earlier in our conversation. Is there anything specific you'd like to know or discuss about artificial intelligence? I'm here to help!", response_metadata={'model': 'mistral:instruct', 'created_at': '2024-07-18T15:48:32.299055Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 844336875, 'load_duration': 8883375, 'prompt_eval_count': 89, 'prompt_eval_duration': 210344000, 'eval_count': 39, 'eval_duration': 619632000}, id='run-7599de75-21e1-4995-95f8-d0f7520cf3b5-0')

## Scenario 2

In [63]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.output_parsers import StrOutputParser

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
#         MessagesPlaceholder(variable_name="messages")
#     ]
# )

# prompt=ChatPromptTemplate.from_template(f"""<s>[INST]You are helpful chatbot who answers all the question as best of your ability.\n\n{MessagesPlaceholder(variable_name="messages")}[/INST]""")
prompt=ChatPromptTemplate.from_template("""<s>[INST]You are helpful chatbot who answers all the question as best of your ability. First Analysis the complete Message and only answer to the Question being asked and do not try add extra information apart what is being asked.\n\nMessage:{message}[/INST]""")
parser = StrOutputParser()
chain=prompt|llm |parser


#### Testing Chain

In [64]:
# chain.invoke({"message":[HumanMessage(content="Hi My name is Nabarun")]})

chain.invoke({"message":"Hi My name is Nabarun"})

" Hello Nabarun, it's nice to meet you! How can I help you today? If you have any questions or need assistance with something, feel free to ask!"

In [65]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# config={"configurable":{"session_id":"chat10"}}
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(chain,
                                                get_session_history,
                                                input_messages_key="message")

config = {"configurable": {"session_id": "chat9"}}
 

response=with_message_history.invoke(
    {"message":"Hi My name is Nabarun and I am Senior Data Scientist at Wipro"},
    config=config
)

response

" Hello Nabarun, it's nice to meet you! As a Senior Data Scientist at Wipro, I can imagine that you work with a variety of fascinating data sets. If there's something specific about your role or the field of data science you'd like to discuss, feel free to ask!"

In [66]:
response=with_message_history.invoke(
    {"message":"what is my name"},
    config=config
)

response

' Your name is Nabarun.'

In [67]:
response=with_message_history.invoke(
    {"message":"what is my Designation"},
    config=config
)

response

' Your designation is Senior Data Scientist.'

## Scenario 3

In [44]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage,HumanMessage

trimmer=trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [45]:
messagess = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messagess)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [46]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.output_parsers import StrOutputParser
 
prompt=ChatPromptTemplate.from_template("""<s>[INST]You are helpful chatbot who answers all the question as best of your ability. First Analysis the complete Message and only answer to the Question being asked and do not try add extra information apart what is being asked.\n\nMessage:{messages}[/INST]""")
prompt


ChatPromptTemplate(input_variables=['messages'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['messages'], template='<s>[INST]You are helpful chatbot who answers all the question as best of your ability. First Analysis the complete Message and only answer to the Question being asked and do not try add extra information apart what is being asked.\n\nMessage:{messages}[/INST]'))])

In [47]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

parser = StrOutputParser()

chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
        |prompt
        |llm
        |parser
    )

In [48]:
chain.invoke({
                "messages":messagess +[
                                        HumanMessage(content="Hi what ice cream I like")
                                    ]
            })

' The user Bob mentioned that he likes vanilla ice cream.'

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# config={"configurable":{"session_id":"chat10"}}
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(chain,
                                                get_session_history,
                                                input_messages_key="messages")

config = {"configurable": {"session_id": "chat9"}}
 

# response=with_message_history.invoke(
#     {"message":"Hi My name is Nabarun and I am Senior Data Scientist at Wipro"},
#     config=config
# )

# response

response=with_message_history.invoke(
    {"messages":"Hi My name is Nabarun and I am Senior Data Scientist at Wipro"},
    
    config=config
)

response

" Hello Nabarun, it's nice to meet you! As a Senior Data Scientist at Wipro, I assume you work with large amounts of data and employ various machine learning techniques to extract valuable insights. How may I assist you today?"

In [50]:
response=with_message_history.invoke(
    {"messages":"what is my name"},
    config=config
)

response

' Your name is Nabarun.'

In [51]:
response=with_message_history.invoke(
    {"messages":"what is my Designation"},
    config=config
)

response

' Your designation is Senior Data Scientist.'

In [52]:
response=with_message_history.invoke(
    {"messages":"what is my name"},
    config=config
)

response

" Your name is Nabarun. I've answered this question in the previous response."

In [14]:
# store

{'chat9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi My name is Nabarun and I am Senior Data Scientist at Wipro'), AIMessage(content=" Hello Nabarun, nice to meet you! It's great that you are a Senior Data Scientist at Wipro. If you have any questions related to data science or need assistance with your work, feel free to ask."), HumanMessage(content='what is my name'), AIMessage(content=' Your name is Nabarun.'), HumanMessage(content='what is my Designation'), AIMessage(content=' Your designation is Senior Data Scientist.'), HumanMessage(content='what is my name'), AIMessage(content=' Your name is Nabarun.')])}

In [15]:
response=with_message_history.invoke(
    {"messages":"what is my Designation"},
    config=config
)

response

' Your designation is Senior Data Scientist.'